# K-Means Image Compression

In this notebook, we demonstrate how to compress an image using the K-Means clustering algorithm. The key idea is to reduce the number of unique colors in the image, thereby decreasing storage size while preserving visual quality.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import importlib
import time
import cv2
from tqdm.notebook import tqdm
from PIL import Image

%matplotlib inline

# Changing Configurations

You can modify some default values to experiment with the notebook. The configurable parameters include:

- **K** (currently set to 16): The number of clusters (colors) the image will be compressed to.  
- **K2** (currently set to 3): An alternative number of clusters for compression.  
- **ImageURL**: You can provide your own **Google Drive** Image URL.


In [2]:
K = 16
K2 = 3
ImageURL = None#Replace None with: '<Your Google Drive URL>'

## Uploading Required Files

The `upload_file` function helps keep the notebook organized. We will use it later to upload `utils.py`, which contains plotting methods, and a sample image (`.png`) for experimentation.


In [3]:
def upload_file(f_name, override=False, external_image=False):
  url = f'https://raw.githubusercontent.com/orwertheim/PublicProjects/main/K%20means%20Image%20compression/{f_name}'

  if external_image:
    if 'drive.google.com/file/d/' in f_name:
            # Extract the file ID
            file_id = f_name.split('/file/d/')[1].split('/')[0]
            print(f"Detected Google Drive file with ID: {file_id}")

            destination = 'image.png'
            if utils.download_from_gdrive(file_id, destination):
                return destination
            else:
                return None
    else:
      print('Not a Google Drive Image URL!')
      return None

  if override or not os.path.exists(f_name):
    timestamp = int(time.time())
    url = f'{url}?t={timestamp}'

  response = requests.get(url)
  if response.status_code == 200:
      with open(f_name, 'wb') as f:
        f.write(response.content)
  else:
    print(f'Error response: {response.status_code}')
  return f_name  # Return the existing filename


In [4]:
upload_file('utils.py', override=True)
import utils

## Understanding the Image Data

As with any machine learning task, we first need to understand our data. Below is the original image we will be working with.


In [ ]:
filename = "colorImage1.png"
if ImageURL is None:
  upload_file('colorImage1.png')
else:
  filename = upload_file(ImageURL, external_image=True)

original_img = plt.imread(filename)
original_img = original_img[:, :, :3]  # Keep only the first 3 channels remove transparancy

# Limit the image resolution to reduce compute time
if(original_img.shape[0] > 250):
  # Calculate the new height (rows)
  new_height = 250
  aspect_ratio = original_img.shape[1] / original_img.shape[0]
  new_width = int(new_height * aspect_ratio)

  # Resize the image
  original_img = cv2.resize(original_img, (new_width, new_height))

utils.display_images([original_img],['Original Image'])

Let's examine the shape and contents of the image data.


In [6]:
print(f'The image shape is: {original_img.shape}')
print(f'The Image is in RGB format, the content of the first five pixel is: {original_img[0,:5]}')

The image shape is: (250, 202, 3)
The Image is in RGB format, the content of the first five pixel is: [[0.87058824 0.25490198 0.23529412]
 [0.9137255  0.30980393 0.30980393]
 [0.99607843 0.40784314 0.45490196]
 [0.8980392  0.34509805 0.41960785]
 [0.92941177 0.40784314 0.5137255 ]]


Now, let's visualize the color distribution of the pixels in our image.


In [ ]:
utils.plot_rgb_3d(original_img, title='Original Image Color Distribution')

## K-Means Algorithm Overview

K-Means clustering operates as follows:

1. **Initialize**: Select $K$ random centroids, each with the same dimensionality as the data points.  
2. **Assignment Step**: Assign each data point to the closest centroid based on Euclidean distance.  
3. **Update Step**: Recompute each centroid as the mean of the assigned data points.  
4. **Repeat**: Iterate the assignment and update steps until convergence.

K-Means is widely used for clustering both structured (tabular) and unstructured data, such as images. In this case, we apply it to image compression by grouping similar pixel colors.


Below is a function to find the closest centroid for each data point.  
**Note**: Vectorizing computations significantly improves efficiency by reducing processing time.


In [8]:
def find_closest_centroids(X, centroids):
    """
    Computes the centroid memberships for every example

    Args:
        X (ndarray): (m, n) Input values
        centroids (ndarray): (K, n) centroids

    Returns:
        idx (array_like): (m,) closest centroids
    """

    # Calculate the squared Euclidean distance between each example and each centroid
    # Broadcasting: X (m, n) and centroids (K, n) => (m, K, n)
    diff = X[:, np.newaxis, :] - centroids  # (m, 1, n) - (K, n) => (m, K, n)
    sq_dist = np.sum(diff**2, axis=2)  # (m, K) squared distances between each example and each centroid

    # Find the index of the closest centroid (the one with the minimum distance)
    idx = np.argmin(sq_dist, axis=1)

    return idx

Recomputing centroids based on their assigned data points.


In [9]:
def compute_centroids(X, idx, K):
    """
    Returns the new centroids by computing the means of the
    data points assigned to each centroid.

    Args:
        X (ndarray):   (m, n) Data points
        idx (ndarray): (m,) Array containing index of closest centroid for each
                       example in X. Concretely, idx[i] contains the index of
                       the centroid closest to example i
        K (int):       number of centroids

    Returns:
        centroids (ndarray): (K, n) New centroids computed
    """

    m, n = X.shape
    centroids = np.zeros((K, n))

    #add each example features to the centroid it belongs to
    np.add.at(centroids, idx, X)

    #count how many examples for each centroid
    counts = np.bincount(idx, minlength=K).reshape(K,1)

    #prevent zero divition
    counts[counts == 0]=1

    #calculate mean
    centroids = centroids/counts

    return centroids


## Cost Function for K-Means

The cost function for K-Means clustering is the **Mean Squared Error (MSE)** between each data point and its assigned centroid. It helps evaluate the quality of the clustering solution.

$$
J = \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} \mathbb{1}(i, k) \cdot \| x_i - \mu_k \|^2
$$

Where:  
- $x_i$ is the $i$-th data point,  
- $\mu_k$ is the $k$-th centroid,  
- $\mathbb{1}(i, k)$ is an indicator function (1 if $x_i$ is assigned to centroid $k$, otherwise 0),  
- $m$ is the total number of data points,  
- $K$ is the number of centroids,  
- $\| x_i - \mu_k \|^2$ is the squared error.


In [10]:
def compute_cost(X, centroids, idx):
    """
    Computes the cost function for K-Means clustering.

    Args:
        X (ndarray): (m, n) Input data, where m is the number of data points and n is the number of features.
        centroids (ndarray): (K, n) Centroids of the clusters, where K is the number of centroids.
        idx (ndarray): (m,) Index array where each element is the index of the centroid assigned to the corresponding data point.

    Returns:
        J (float): The computed cost (sum of squared errors) for the given centroids and assignments.
    """

    # Get the number of data points and centroids
    m = X.shape[0]

    # Efficiently compute the squared distances using broadcasting
    # X (m, n) -> subtract from the corresponding centroid (centroids[idx] will have shape (m, n))
    # The subtraction is done element-wise and then squared
    diff = X - centroids[idx]  # Shape: (m, n)
    squared_diff = np.sum(diff**2, axis=1)  # Sum the squared differences along the feature axis, result is shape (m,)

    # The cost is the average of the squared distances
    J = np.sum(squared_diff) / m

    return J

## Enhanced K-Means Implementation

The K-Means implementation below incorporates two key improvements:

1. **Smart Initialization**: Instead of random selection, centroids are chosen from the data points, improving convergence speed.  
2. **Multiple Runs & Best Selection**: The algorithm is executed multiple times with different initializations, selecting the best result based on the cost function.

Choosing an appropriate $K$ can be done using the **elbow method**, but in many cases—including ours—it is practical to experiment with different values and choose the one that works best.


In [11]:
def run_kMeans(X, initial_centroids, max_iters=10, prints=False):
    """
    Runs the K-Means algorithm on data matrix X, where each row of X
    is a single example
    """

    # Initialize values
    m, n = X.shape
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    idx = np.zeros(m)
    # Run K-Means
    last_cost=-1

    #for i in tqdm(range(max_iters), desc="K-Means iterations", unit="item"):
    for i in range(max_iters):
        # For each example in X, assign it to the closest centroid
        idx = find_closest_centroids(X, centroids)

        # Given the memberships, compute new centroids
        centroids = compute_centroids(X, idx, K)
        last_cost=compute_cost(X,centroids, idx)
        #Output progress
        if prints:
          print(f"K-Means iteration: {i}/{max_iters-1}, cost: {last_cost}")
    return centroids, idx, last_cost

Selecting random centroids from the data points.


In [12]:
def kMeans_init_centroids(X, K):
    """
    This function initializes K centroids that are to be
    used in K-Means on the dataset X

    Args:
        X (ndarray): Data points
        K (int):     number of centroids/clusters

    Returns:
        centroids (ndarray): Initialized centroids
    """

    # Randomly reorder the indices of examples
    randidx = np.random.permutation(X.shape[0])

    # Take the first K examples as centroids
    centroids = X[randidx[:K]]

    return centroids

Reshaping our image into a matrix, where each row represents a pixel and consists of three columns corresponding to the R, G, and B color channels.

In [13]:
X_img = np.reshape(original_img, (original_img.shape[0] * original_img.shape[1], 3))

We will run K-Means with 300 different initializations for both $K = 16$ and $K = 3$.

In [ ]:
# Run your K-Means algorithm on this data
# You should try different values of K and max_iters here

max_iters = 10

initializations_count = 300
pairs = list(zip([K, K2] * initializations_count, list(range(initializations_count)) * 2))
centroids_ar = []
cost_ar = []
idx_ar = []

best_k2_centroid = None
best_k2_cost = float('inf')
best_k2_idx = None

for k_value, init in tqdm(pairs, desc="K-Means Runs", unit="Run"):
  # Using the function you have implemented above.
  initial_centroids = kMeans_init_centroids(X_img, k_value)
  # Run K-Means - this can take a couple of minutes depending on K and max_iters
  centroids, idx, cost = run_kMeans(X_img, initial_centroids, max_iters)
  if k_value == K:
    centroids_ar.append(centroids); idx_ar.append(idx); cost_ar.append(cost)
  elif cost < best_k2_cost:
    best_k2_cost = cost
    best_k2_centroid = centroids
    best_k2_idx = idx



To understand the importance of running K-Means multiple times, we will plot the mean squared distance of each pixel from its assigned centroid. This will help us visualize the variance across different runs.

In [ ]:
y_range = (np.min(cost_ar)-0.001, np.max(cost_ar)+0.001)
utils.plot_costs(cost_ar, y_label="Mean Squared Distance of Pixels to Their Centroids", x_label='K-Means Run (Initialization Attempt)', title='Pixel Similarity to Centroids Across Multiple K-Means Runs', y_lim=y_range)

We will now visualize the pixel distributions for the original image, the best and worst runs with $K = 16$, and the best run with $K = 3$. This comparison highlights the trade-off between lower $K$ values, which lead to higher compression, and higher $K$ values, which better preserve image quality.

In [ ]:
best_centroids = centroids_ar[np.argmin(cost_ar)]
best_centroids_idx = idx_ar[np.argmin(cost_ar)]

worst_centroids = centroids_ar[np.argmax(cost_ar)]
worst_centroids_idx = idx_ar[np.argmax(cost_ar)]

#utils.plot_kMeans_RGB(X_img, best_centroids, best_centroids_idx, centroids2 = best_k2_centroid, idx2 = best_k2_idx, sample_size=1000000)
utils.plot_kMeans_RGB(X_img, [best_centroids, worst_centroids, best_k2_centroid], [best_centroids_idx, worst_centroids_idx, best_k2_idx], ['Original colors',f'Pixel Colors Based on best run with {len(best_centroids)} Centroids',f'Pixel Colors Based on worst run with {len(best_centroids)} Centroids', f'Pixel Colors Based on {len(best_k2_centroid)} Centroids'] , sample_size=1000000)

Let's examine the reconstructed images to understand the importance of running K-Means multiple times. By comparing the best and worst runs, we can clearly see the differences in image quality.

In [ ]:

best_image = np.reshape(best_centroids[best_centroids_idx],original_img.shape)



worst_image = np.reshape(worst_centroids[worst_centroids_idx],original_img.shape)


k2_image = np.reshape(best_k2_centroid[best_k2_idx],original_img.shape)

utils.display_images([original_img, best_image, worst_image, k2_image],['Original Image', f'Best Clustering Image For {K} colors', f'Worst Clustering Image For {K} colors', f'Best Clustering Image For {K2} colors'])

Below are the colors selected for the best run with $K = 16$.

In [ ]:
utils.show_centroid_colors(best_centroids)

Below are the colors selected for the worst run with $K = 16$.

In [ ]:
utils.show_centroid_colors(worst_centroids)

Below are the colors selected for the best run with $K = 3$.

In [ ]:
utils.show_centroid_colors(best_k2_centroid)